## Import libraries

In [100]:
import json
import sys
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import numpy as np
import pandas as pd
import keras
from pandas.io.json import json_normalize
import re
import gensim
import logging
import os
LabeledSentence = gensim.models.doc2vec.LabeledSentence

## Neural Network Models

### Build model for Words

### Filter Data for  Words

In [101]:
from nltk.tokenize import RegexpTokenizer
def clean_data(data):
    tokenizer = RegexpTokenizer(r'\w+')
    data_processed = re.sub(r'\b\w{1,2}\b', '', str(data))
    words = tokenizer.tokenize(data_processed)
    operators = ['none', 'name', 'value']
    stop = set(stopwords.words('english') + operators)
    cleaned_data =[]
    for word in words:
        if word.lower() not in stop and word.lower() not in operators:
            cleaned_data.append(word)
    return cleaned_data

### Read Json

In [102]:
data = []
paths = ['ashfaq_people_8k.json']
type = []
good = 0
bad = 0
total = 0
labels1 = []
# Open file process  and adding source data row for _BULK operations
for file in paths:
    with open(file) as f:
        for line in f:
            sys.stdout.write("\r" + 'All files: ' + (str(total) + '  ' + 'Good Files: ' + str(good) + '  ' + \
                                                     'Bad Files: ' + str(bad)))
            try:
                data.append(clean_data(json.loads(line)))
                labels1.append(str(json.loads(line)['_id']['$oid']))
                good += 1
                total += 1
            except:
                total += 1
                bad += 1
                continue

All files: 7927  Good Files: 7927  Bad Files: 0

###  Train an unsupervised model 

In [103]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gensim.models.Word2Vec(data, size=200, window=5, min_count=1, workers=4)

2017-05-03 14:43:43,023 : INFO : collecting all words and their counts
2017-05-03 14:43:43,024 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-03 14:43:48,014 : INFO : collected 488664 word types from a corpus of 23949638 raw words and 7928 sentences
2017-05-03 14:43:48,015 : INFO : Loading a fresh vocabulary
2017-05-03 14:43:52,277 : INFO : min_count=1 retains 488664 unique words (100% of original 488664, drops 0)
2017-05-03 14:43:52,278 : INFO : min_count=1 leaves 23949638 word corpus (100% of original 23949638, drops 0)
2017-05-03 14:43:53,411 : INFO : deleting the raw counts dictionary of 488664 items
2017-05-03 14:43:53,443 : INFO : sample=0.001 downsamples 63 most-common words
2017-05-03 14:43:53,443 : INFO : downsampling leaves estimated 21664475 word corpus (90.5% of prior 23949638)
2017-05-03 14:43:53,444 : INFO : estimated required memory for 488664 words and 200 dimensions: 1026194400 bytes
2017-05-03 14:43:55,090 : INFO : resetting layer w

In [104]:
model.train(data)

2017-05-03 14:47:02,527 : INFO : training model with 4 workers on 488664 vocabulary and 200 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2017-05-03 14:47:02,528 : INFO : expecting 7928 sentences, matching count from corpus used for vocabulary survey
2017-05-03 14:47:03,545 : INFO : PROGRESS: at 0.82% examples, 881661 words/s, in_qsize 7, out_qsize 0
2017-05-03 14:47:04,542 : INFO : PROGRESS: at 1.70% examples, 926430 words/s, in_qsize 8, out_qsize 0
2017-05-03 14:47:05,543 : INFO : PROGRESS: at 2.63% examples, 943804 words/s, in_qsize 7, out_qsize 0
2017-05-03 14:47:06,561 : INFO : PROGRESS: at 3.52% examples, 947943 words/s, in_qsize 8, out_qsize 1
2017-05-03 14:47:07,564 : INFO : PROGRESS: at 4.43% examples, 949869 words/s, in_qsize 7, out_qsize 0
2017-05-03 14:47:08,568 : INFO : PROGRESS: at 5.24% examples, 935392 words/s, in_qsize 7, out_qsize 0
2017-05-03 14:47:09,571 : INFO : PROGRESS: at 6.09% examples, 929926 words/s, in_qsize 7, out_qsize 0
2017-05-03 14:47:10,58

107489984

### Model has been trained lets run some test 

In [107]:
model.most_similar(positive = ['58ea248724dce09f41209ddc'], topn=5)

[('58ea248724dce09f41209d09', 0.8010620474815369),
 ('58ea248624dce09f41209a69', 0.7969576120376587),
 ('58ea248524dce09f41209855', 0.791680097579956),
 ('58ea248524dce09f4120996b', 0.7904684543609619),
 ('58ea248524dce09f41209976', 0.788763165473938)]

In the above test, we asked the model to find some result similiar to '58ea248724dce09f41209ddc' and it gives the following output

In [111]:
model.most_similar(positive = ['lname'], topn=10)

[('anotherAddresses', 0.7202301025390625),
 ('homeValue', 0.5573374629020691),
 ('cmilak', 0.49501287937164307),
 ('BUTCHER', 0.4755026400089264),
 ('ALBERTA', 0.4445800483226776),
 ('going_on_for_christ', 0.4381500780582428),
 ('jane', 0.437492311000824),
 ('HAE', 0.4334526062011719),
 ('Muzio', 0.4316481947898865),
 ('maxp', 0.4258100688457489)]

outputs for keyword lname, 6/10 are similiar

In [123]:
model.most_similar(positive = ['city'], topn=10)

[('12402285000', 0.49995890259742737),
 ('34756', 0.47860804200172424),
 ('BERLIN', 0.4752332866191864),
 ('S9tWi5sGSehIMtu_', 0.47192835807800293),
 ('WHITESBORO', 0.4669457674026489),
 ('33475', 0.4628153443336487),
 ('2073754465', 0.4476548731327057),
 ('SEBASTIAN', 0.44685837626457214),
 ('4809269740', 0.4468235969543457),
 ('SCARBOROUGH', 0.4458960294723511)]

### Build Model for sentences : Work in Progress

### Filtering

In [124]:
from nltk.tokenize import RegexpTokenizer
def clean_data(data):
    tokenizer = RegexpTokenizer(r'\w+')
    data_processed = re.sub(r'\b\w{1,2}\b', '', str(data))
    words = tokenizer.tokenize(data_processed)
    operators = ['none', 'name', 'value']
    stop = set(stopwords.words('english') + operators)
    cleaned_data =[]
    for word in words:
        if word.lower() not in stop and word.lower() not in operators:
            cleaned_data.append(word)
    return ' '.join(cleaned_data)

### Read Json

In [125]:
data = []
paths = ['ashfaq_people_8k.json']
type = []
good = 0
bad = 0
total = 0
labels1 = []
# Open file process  and adding source data row for _BULK operations
for file in paths:
    with open(file) as f:
        for line in f:
            sys.stdout.write("\r" + 'All files: ' + (str(total) + '  ' + 'Good Files: ' + str(good) + '  ' + \
                                                     'Bad Files: ' + str(bad)))
            try:
                data.append(clean_data(json.loads(line)))
                labels1.append(str(json.loads(line)['_id']['$oid']))
                good += 1
                total += 1
            except:
                total += 1
                bad += 1
                continue

All files: 7927  Good Files: 7927  Bad Files: 0

In [126]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])

In [127]:
it = LabeledLineSentence(data, labels1)

In [59]:
model = gensim.models.Doc2Vec(size=300, window=10, min_count=5, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
for epoch in range(10):
    model.train(it)
    model.alpha -= 0.002 # decrease the learning rate
    model.min_alpha = model.alpha # fix the learning rate, no deca
    model.train(it)

2017-05-03 12:26:31,358 : INFO : collecting all words and their counts
2017-05-03 12:26:31,360 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-03 12:26:32,194 : INFO : collected 104754 word types and 1000 unique tags from a corpus of 1000 examples and 3031597 words
2017-05-03 12:26:32,196 : INFO : Loading a fresh vocabulary
2017-05-03 12:26:32,294 : INFO : min_count=5 retains 24705 unique words (23% of original 104754, drops 80049)
2017-05-03 12:26:32,295 : INFO : min_count=5 leaves 2897168 word corpus (95% of original 3031597, drops 134429)
2017-05-03 12:26:32,367 : INFO : deleting the raw counts dictionary of 104754 items
2017-05-03 12:26:32,374 : INFO : sample=0.001 downsamples 63 most-common words
2017-05-03 12:26:32,376 : INFO : downsampling leaves estimated 2580829 word corpus (89.1% of prior 2897168)
2017-05-03 12:26:32,378 : INFO : estimated required memory for 24705 words and 300 dimensions: 73044500 bytes
2017-05-03 12:26:32,469 : INFO 

In [60]:
model.save("doc2vec.model")

2017-05-03 12:33:49,812 : INFO : saving Doc2Vec object under doc2vec.model, separately None
2017-05-03 12:33:49,814 : INFO : not storing attribute syn0norm
2017-05-03 12:33:49,815 : INFO : not storing attribute cum_table
2017-05-03 12:33:50,170 : INFO : saved doc2vec.model


In [64]:
print model.most_similar("oid")

[('searchResult', 0.480995237827301), ('Filtering', 0.39368563890457153), ('Malware', 0.39008861780166626), ('630d5ca9', 0.3804160952568054), ('TOWNHOMES', 0.3767187297344208), ('142e', 0.3757261633872986), ('BILLIONS', 0.37459099292755127), ('42633655', 0.3699648976325989), ('SLAP', 0.36792731285095215), ('HOMEOWNERS', 0.36769628524780273)]


In [52]:
labels[2]

'58ea248524dce09f41209712'

## Test Sentence Model

In [88]:
paths = ['ashfaq_people_8k.json']
test_data = []
test_labels = []
good = 0
# Open file process  and adding source data row for _BULK operations
for file in paths:
    with open(file) as f:
        for line in f:
            good += 1
            if good ==1001:
                results = clean_data(json.loads(line))
                with open('/home/siddhant/test.txt','w') as f:
                    for result in results:
                        f.write(result + ' \n')
                    f.seek(0)
                break

In [93]:
model.accuracy('test.txt')

2017-05-03 13:19:19,398 : INFO : skipping invalid line #1 in test.txt
2017-05-03 13:19:19,398 : INFO : skipping invalid line #2 in test.txt
2017-05-03 13:19:19,399 : INFO : skipping invalid line #3 in test.txt
2017-05-03 13:19:19,400 : INFO : skipping invalid line #4 in test.txt
2017-05-03 13:19:19,400 : INFO : skipping invalid line #5 in test.txt
2017-05-03 13:19:19,401 : INFO : skipping invalid line #6 in test.txt
2017-05-03 13:19:19,402 : INFO : skipping invalid line #7 in test.txt
2017-05-03 13:19:19,402 : INFO : skipping invalid line #8 in test.txt
2017-05-03 13:19:19,403 : INFO : skipping invalid line #9 in test.txt
2017-05-03 13:19:19,404 : INFO : skipping invalid line #10 in test.txt
2017-05-03 13:19:19,404 : INFO : skipping invalid line #11 in test.txt
2017-05-03 13:19:19,405 : INFO : skipping invalid line #12 in test.txt
2017-05-03 13:19:19,405 : INFO : skipping invalid line #13 in test.txt
2017-05-03 13:19:19,406 : INFO : skipping invalid line #14 in test.txt
2017-05-03 13:1

[{'correct': [], 'incorrect': [], 'section': u'_id'},
 {'correct': [], 'incorrect': [], 'section': 'total'}]

## Implementation in Keras 

In [2]:
# word_vectors_pd = pd.DataFrame({'data':data,'labels':labels})

In [139]:
index_dict = {k: g["labels"].tolist() for k,g in index_dict_pd.groupby("index")}

In [150]:
word_vectors = {k: g["data"].tolist() for k,g in word_vectors_pd.groupby("labels")}

In [331]:
index_dict_pd.to_csv("index.csv")
word_vectors_pd.to_csv("words.csv")

In [2]:
index_dict_pd = pd.read_csv("index.csv")
word_vectors_pd = pd.read_csv("words.csv")

In [79]:
word_vectors_pd.head()

,Unnamed: 0,data,labels
0,0,"[""u'_id"", 'u', 'oid', ""u'58ea248524dce09f41209...",58ea248524dce09f41209710
1,1,"[""u'_id"", 'u', 'oid', ""u'58ea248524dce09f41209...",58ea248524dce09f41209711
2,2,"[""u'_id"", 'u', 'oid', ""u'58ea248524dce09f41209...",58ea248524dce09f41209712
3,3,"[""u'_id"", 'u', 'oid', ""u'58ea248524dce09f41209...",58ea248524dce09f41209713
4,4,"[""u'_id"", 'u', 'oid', ""u'58ea248524dce09f41209...",58ea248524dce09f41209714


In [151]:
for word,index in index_dict.items(): 
    print index,word
    break

['58ea248524dce09f41209710'] 0


In [72]:
str(json.loads(line)['_id']['$oid'])


'58ea249124dce09f4120b61d'

In [20]:
from keras.models import Model
from keras.layers import Input,Embedding,Dense, Flatten, Conv2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

nb_features = 1000
sequence_length = 100
seq_input = Input(shape=(1000,))

# # embedded_input = Embedding(nb_features,32,input_length=sequence_length)(seq_input)
# dense = Dense(500,activation='relu')(seq_input)
# flatten_vectors = Flatten()(dense)

hidden_layer = Dense(100,activation='relu')(seq_input)
output_layer = Dense(1,activation='softmax')(hidden_layer)

weight_checkpoint = ModelCheckpoint('best_validation_accuracy.h5',monitor='val_loss',mode='auto')

model = Model(input=seq_input, output = output_layer)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 100,201.0
Trainable params: 100,201.0
Non-trainable params: 0.0
_________________________________________________________________


/home/siddhant/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [3]:
# model.fit(np.array(new_train),np.array(indexes),
#           nb_epoch=5,batch_size=64,
#           verbose=2)

## Implementation in Tensorflow : might not work

In [ ]:
import tensorflow as tf
data = []
labels = []
test_data = []
test_labels = []
feed = (data,label)
batch_size = 100
sess = tf.InteractiveSession()
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(20000):
    batch = feed[index : index + 100]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print((i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
print(accuracy.eval(feed_dict={x: feed[0], y_: feed[1], keep_prob: 1.0}))